In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Preprocesamiento inicial de la tabla de datos sociodemográficos**

####**Descripción:** 
Abre el archivo original de datos sociodemográfico `Sociodemografico.zip`  y le asigna a cada columna el nombre y el tipo de datos de Pandas (*Pandas datatypes*) más adecuado, con el fin de  ahorrar memoria. Guarda esta tabla de datos en formato binario `parquet`.

####**Requerimientos:**
- Diccionario `datatypes_dictionaries/dtypes_sociodemo.json`, que mapea cada columna de la tabla de datos sociodemográficos al *datatype* de *Pandas* más adecuado.

-  Archivo `raw_data/Sociodemografico.zip` original (versión enviada por el ICBF el 7 de julio de 2021), con la tabla de datos nutricionales en formato `csv` (separador `|`, codificación `cp-1252`), sumistrada por el ICBF bajo el nombre [`ICBF_Reincidenciadesnutricion_Sociodemografico_Sisbeniv4.csv`](https://icbfgob-my.sharepoint.com/:u:/g/personal/johanna_lozano_icbf_gov_co/EQFdZ0SY79JChRAD214FelkBYS73R46RysSk8iPMrmLGFw?e=iuQbDY).


####**Salidas:**
- Tabla de datos `preprocessed_data/sociodemo.parquet` con los datos sociodemográficos de cada beneficiario del SISBÉN (~6 millones de filas, una por cada fila en la tabla original).

- Archivo `log_files/log_sociodemo_preprocesamiento.txt` con el reporte de cambios realizados.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import json
import re
from utils import replace_log

### **Lectura del archivo**
Toma el archivo `Sociodemografico.zip`, de 140 MB, comprimido del archivo original llamado [`ICBF_Reincidenciadesnutricion_Sociodemografico_Sisbeniv4.csv`](https://icbfgob-my.sharepoint.com/:u:/g/personal/johanna_lozano_icbf_gov_co/EQFdZ0SY79JChRAD214FelkBYS73R46RysSk8iPMrmLGFw?e=iuQbDY) (originalmente comprimido en formato `rar`, tal como se descargó del repositorio de One Drive, enviado a nosotros por el ICBF el 7 de julio de 2021),  codificado en formato `cp-1252`, del directorio `raw_data` y lo subdivide en `n` = 100 fragmentos, de ~ 60076 registros cada uno. Requiere el diccionario de datos del archivo `dtypes_tomas.json` con la estructura de datos predefinida.


In [ ]:
t0 = time.time()
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
with open('datatypes_dictionaries/cols_ord.json', 'r') as file:
  cols_ord = json.load(file)

# Lee el archivo
soc = pd.read_csv('raw_data/Sociodemografico.zip', sep ='|', header=0,
                  names=dtypes_soc.keys(), encoding='cp1252')
print(f'Archivo leído: {time.time() - t0:.2f} s.')
soc.info()

Archivo leído: 41.56 s.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 68 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Cod_clase                  int64  
 1   Ind_grupo_sisben_4         object 
 2   Ind_nivel_sisben_4         int64  
 3   Tip_vivienda               int64  
 4   Ind_tiene_energia          int64  
 5   Ind_tiene_alcantarillado   int64  
 6   Ind_tiene_gas              int64  
 7   Ind_tiene_recoleccion      int64  
 8   Ind_tiene_acueducto        int64  
 9   n_hogares_vivienda         int64  
 10  Tip_ocupa_vivienda         int64  
 11  Ind_tiene_cocina           int64  
 12  Ind_tiene_nevera           int64  
 13  Ind_tiene_lavadora         int64  
 14  Ind_tiene_pc               int64  
 15  Ind_tiene_internet         int64  
 16  n_personas_hogar           int64  
 17  Sexo                       float64
 18  Ind_ninguna_discapac       float64
 19  Tip_seg_social    

### **Preprocesamiento**
- Reemplazar algunas fechas de nacimiento futuras (`'FechaNacimiento'` > año 2800, seguramente errores de digitación) por sus equivalentes más probables, para permitir la asignación del tipo `datetime64[ns]`.

In [ ]:
# Eliminar fechas absurdas (año > 2800)
log = list()
soc['FechaNacimiento'] = replace_log(soc, col='FechaNacimiento', col_id='Id', 
                                     map={'^2[89]': '20'}, log=log)

In [ ]:
# Asigna a cada variable según su tipo (modo ahorro de memoria)
# Contiene tipos de datos incompatibles con parquet
soc['Sexo'] = soc['Sexo'].astype('Int8')
soc = soc.astype(dtypes_soc)
# Establecer el orden de las categorías de las variables ordinales según su orden natural   
for col in cols_ord['soc']:
  soc[col] = soc[col].cat.as_ordered()
soc.info()
print(f'Preprocesado: {time.time() - t0:.2f} s.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 68 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   Cod_clase                  category      
 1   Ind_grupo_sisben_4         category      
 2   Ind_nivel_sisben_4         category      
 3   Tip_vivienda               category      
 4   Ind_tiene_energia          category      
 5   Ind_tiene_alcantarillado   category      
 6   Ind_tiene_gas              category      
 7   Ind_tiene_recoleccion      category      
 8   Ind_tiene_acueducto        category      
 9   n_hogares_vivienda         UInt8         
 10  Tip_ocupa_vivienda         category      
 11  Ind_tiene_cocina           category      
 12  Ind_tiene_nevera           category      
 13  Ind_tiene_lavadora         category      
 14  Ind_tiene_pc               category      
 15  Ind_tiene_internet         category      
 16  n_personas_hogar           UInt8    

### **Guardado**
Guarda el dataframes preprocesado en el archivo en formato binario comprimido `sociodemo.parquet` (143 MB), que permite guardar la estructura de los tipos de datos, en el directorio `preprocessed_data`.

In [ ]:
# Escribe un archivo con los cambios generados
with open('log_files/log_sociodemo_preprocesamiento.txt', 'w') as log_file:
  log_file.write('Preprocesamiento_Sociodemográfico:\n')
  for line in log:
    log_file.write('\t' + line + '\n')
 
# Guardar el dataframe preprocesado en un archivo parquet
soc.to_parquet('preprocessed_data/sociodemo.parquet')
print(f'Nuevo archivo guardado: {time.time() - t0:.2f} s.')

Nuevo archivo guardado: 76.10 s.
